In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px

In [ ]:
bs_dt = pd.read_csv('../input/bank-marketing/bank-additional-full.csv', header = 0, sep=';')

In [ ]:
def hd(*args):
    k = args
    if(len(k) > 0):
        return k[0].head()
    else:
        return bs_dt.head()

In [ ]:
bs_dt = bs_dt[~bs_dt.duplicated()]

In [ ]:
bs_dt.info()

In [ ]:
for i in  bs_dt.columns.values:
    if((bs_dt[i].nunique() < 25) & ( bs_dt[i].dtype not in [np.int64, np.float64])):
        bs_dt[i] = bs_dt[i].astype('category')

In [ ]:
lt =[]
lt.extend(['a','b'])
print(lt)

In [ ]:
def make_bar_plot_with_target(groupvar, *args):
    group_list = []
    group_list.extend([groupvar, 'y','age'])
    df = bs_dt.groupby([groupvar, 'y']).count().reset_index()[group_list].rename(columns = {'age':'count','y':'convert'})
    figure = px.bar(df, x = groupvar, y = 'count', barmode = args[1], color = 'convert', log_y = args[0])
    return figure

In [ ]:
px.bar(bs_dt.groupby(['job','y']).count().reset_index()[['job','y','age']].sort_values(by = 'age').rename(columns = {'age':'count'}),log_y = True, y='count',x = 'job',color = 'y', barmode = 'group',opacity = 0.6)

In [ ]:
figures_var = {}
for col in [i for i in bs_dt.columns.values.tolist()[:-1] if bs_dt[i].dtype.name == 'category']:
    figures_var[col] = make_bar_plot_with_target(col, True, 'relative')

In [ ]:
[i for i in bs_dt.columns.values.tolist()[:-1] if bs_dt[i].dtype.name == 'category']

In [ ]:
figures_var['job']

In [ ]:
figures_var['marital']

In [ ]:
figures_var['education']

In [ ]:
figures_var['housing']

In [ ]:
figures_var['loan']

In [ ]:
figures_var['contact']

In [ ]:
figures_var['month']

In [ ]:
figures_var['day_of_week']

In [ ]:
figures_var['poutcome']

In [ ]:
bs_dt.rename(columns = {'y':'target'}, inplace = True)

In [ ]:
bs_dt.columns

In [ ]:
target_map = {
    'no':0,
    'yes':1
}
bs_dt['target'] = bs_dt['target'].map(target_map)

In [ ]:
for col in [i for i in bs_dt.columns.values.tolist()[:-1] if bs_dt[i].dtype.name == 'category']:
    map_var = dict(zip(bs_dt[col].unique().tolist(), range(len(bs_dt[col].unique().tolist()))))
    bs_dt[col] = bs_dt[col].map(map_var)

In [ ]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()
X = bs_dt[bs_dt.columns.values.tolist()[:-1]]
y = bs_dt[["target"]]
logreg.fit(X,y)
print(logreg.coef_)
print(logreg.intercept_)

In [ ]:
def auc(variables, target, basetable):
    X = basetable[variables]
    y = basetable[target]
    logreg = linear_model.LogisticRegression()
    logreg.fit(X, y)
    predictions = logreg.predict_proba(X)[:,1]
    auc = roc_auc_score(y, predictions)
    return(auc)

def next_best(current_variables,candidate_variables, target, basetable):
    best_auc = -1
    best_variable = None
    for v in candidate_variables:
        auc_v = auc(current_variables + [v], target, basetable)
        if auc_v >= best_auc:
            best_auc = auc_v
            best_variable = v
    return best_variable

In [ ]:
current_var = []
candidate_var = [i for i in bs_dt.columns.values.tolist()[:-1]]
next_variable = next_best(current_var, candidate_var, 'target', bs_dt)
print(next_variable)

In [ ]:
candidate_variables = [i for i in bs_dt.columns.values.tolist()[:-1]]
current_variables = []
target = "target"
max_number_variables = 5
number_iterations = min(max_number_variables, len(candidate_variables))
for i in range(0,number_iterations):
    next_var = next_best(current_variables,candidate_variables,target,bs_dt)
    current_variables = current_variables + [next_var]
    candidate_variables.remove(next_var)
print(current_variables)

In [ ]:
auc_values = []
variables_evaluate = []
for v in [i for i in bs_dt.columns.values.tolist()[:-1]]:
    variables_evaluate.append(v)
    auc_value = auc(variables_evaluate, "target", bs_dt)
    auc_values.append(auc_value)

In [ ]:
px.line(auc_values)

In [ ]:
y = bs_dt['target']

In [ ]:
X = bs_dt.drop(columns = 'target')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42, stratify = y)

In [ ]:
def auc_s(X, y):
#     X = basetable[variables]
#     y = basetable[target]
   
    logreg = linear_model.LogisticRegression()
    logreg.fit(X, y)
    predictions = logreg.predict_proba(X)[:,1]
    auc = roc_auc_score(y, predictions)
    return(auc)

var_set = []
auc_all = dict()
for col in X_train.columns.values:
    var_set.append(col)
    auc_all['{0}_feaures'.format(len(var_set))] = auc_s(X_train[var_set], y_train)
auc_all

In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
def pipeli(X_Tr, y_Tr, X_Te, y_Te):
    pipe.fit(X_Tr, y_Tr)
    predictions = pipe.predict_proba(X_Te)[:,1]
    return roc_auc_score(y_Te, predictions)

var_set_p = []
score_all_p = dict()
roc_curv_plot = pd.DataFrame(columns = ['feature_count','ns_fpr','ns_tpr'])
for index, col in enumerate(X_train.columns.values):
    var_set_p.append(col)
    score_all_p['{0}_feaures'.format(len(var_set_p))] = pipeli(X_train[var_set_p], y_train, X_test[var_set_p], y_test)
    roc_curv_plot.loc[index, 'feature_count'] = '{0}_feaures'.format(len(var_set_p))
    predictions = pipe.predict_proba(X_test[var_set_p])[:,1]
    roc_curv_plot.loc[index,'ns_fpr'] = [roc_curve(y_test, predictions)[0]][0]
    roc_curv_plot.loc[index,'ns_tpr'] = [roc_curve(y_test, predictions)[1]][0]
score_all_p

In [ ]:
roc_curv_plot

In [ ]:
fig = px.line(roc_curv_plot, y = roc_curv_plot.ns_fpr[0], x = roc_curv_plot.ns_tpr[0], height = 800, title = 'AUC Curve', labels = {'x':'False Positives', 'y':'True Positives'})
for i in roc_curv_plot.iterrows():
    fig.add_scatter(x = i[1].ns_fpr, y = i[1].ns_tpr, mode='lines')
fig.add_shape(type="line",
    x0=0, y0=0, x1=1, y1=1,
    line=dict(
        color="Black",
        width=4,
        dash="dot",
    )
)
fig.show()

In [ ]:
logreg1 = linear_model.LogisticRegression()
logreg1.fit(X_train, y_train)
predictions1 = logreg1.predict_proba(X_test)

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt
skplt.metrics.plot_cumulative_gain(y_test, predictions1)
plt.show()

In [ ]:
skplt.metrics.plot_lift_curve(y_test, predictions1)
plt.show()